In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup   
from urllib.parse import urlparse, parse_qs
import re

In [3]:
def get_df(url,search_word):
    prod_title=[]
    prices=[]
    prod_ratings=[]
    description=[]
    num_ratings=[]
    num_reviews=[]
    prod_id=[]
    images=[]
    seller=[]
    seller_ratings=[]

    req=requests.get(url)
    print(req)
    content=BeautifulSoup(req.content,"html.parser")
    box=content.find("div",class_='_1YokD2 _2GoDe3')
    
    data=box.find_all('div',{'class':'_4ddWXP'})
    links=[]
    start_link="https://www.flipkart.com"
    
    #TITLE
    for items in data:
        rest_link=items.find('a')['href']
        name=items.find('a',attrs={'class':'s1Q9rs'})
        prod_title.append(name.text)
        links.append(start_link+rest_link)
        
    #PRODUCT RATINGS
    for items in data:
        rating_div = items.find('div', class_='_3LWZlK')
        if rating_div:
            prod_ratings.append(rating_div.text.strip())
        else:
            prod_ratings.append('NA')

    num_rev=[]
    num_rat=[]
    
    for i in links:
        r=requests.get(i)
        soup=BeautifulSoup(r.text,"html.parser")
        
        #PRODUCT ID
        parsed_url = urlparse(i)
        query_params = parse_qs(parsed_url.query)
        pid_param = query_params.get('pid', None)
        if pid_param:
            pid_value = pid_param[0]
            prod_id.append(str(pid_value))
        else:
            prod_id.append('NA')
        
        
        #PRICES
        pri=soup.find_all('div',class_='_30jeq3 _16Jk6d')
        if pri:
            for i in pri:
                pri_only=i.text
                prices.append(pri_only)
        else:
            prices.append("NA")
        
        #IMAGES
        uls = soup.find_all('ul',class_='_3GnUWp')
        if uls:
            # Iterate over each unordered list
            for ul in uls:
                image_list=[]

                # Find all img tags within the unordered list
                imgs = ul.find_all('img')

                # Extract the source (src) attribute of each image and print it
                for img in imgs:
                    image_list.append(img['src'])
                images.append(image_list)

        else:
            imgs=soup.find_all('img',class_='_396cs4 _2amPTt _3qGmMb')
            image_list2=[]

            for img in imgs:
                image_list2.append(img['src'])
            images.append(image_list2)
        
        
        #DESCRIPTION
        desc=soup.find_all('div',class_='_2cM9lP')
        if desc:
            for i in desc:
                desc_only=i.text
                description.append(desc_only)
        else:
            description.append("NA")
        
        #SELLER
        sel=soup.find('div',class_='_1RLviY')
        if sel:
            second_span = sel.find('span').find_next('span') 
            seller.append(second_span.text.strip())
        else:
            seller.append("NA")
        
        #SELLER RAINGS
        selrat=soup.find_all('div',class_='_3LWZlK _1D-8OL')
        if selrat:
            for i in selrat:
                selrat_only=i.text
                seller_ratings.append(selrat_only)
        else:
            seller_ratings.append('No rating')
        
        #REVIWES & RATINGS
        contents= soup.find_all('div', {"class": "row _2afbiS"})
        if contents:
            for content in contents:
                span = content.find('span')
                if span and 'Reviews' in span.text:
                    reviews_text = span.text
                    num_rev.append(reviews_text)
                if span and 'Ratings' in span.text:
                    ratings_text = span.text
                    num_rat.append(ratings_text[:-2])
        else:
            num_rev.append('0 Reviews')
            num_rat.append('0 Ratings')
            
        
    
    # Check lengths of lists
    print("Length of prod_id:", len(prod_id))
    print("Length of prices:", len(prices))
    print("Length of ratings:", len(prod_ratings))
    print("Length of images:", len(images))
    print("Length of description:", len(description))
    print("Length of seller:", len(seller))
    print("Length of seller_ratings:", len(seller_ratings))
    print("Length of num_reviews:", len(num_rev))
    print("Length of num_ratings:", len(num_rat))
    print("\n")
    
    df=pd.DataFrame({"Search Word":search_word,"Product Title":prod_title,"Prices":prices,"Rating":prod_ratings,"Description":description,"Num of Ratings":num_rat,"Num of Reviews":num_rev,"Product Id":prod_id,"Images":images,"Seller":seller,"Seller Ratings":seller_ratings})
    df

    return df

In [4]:
base_url="https://www.flipkart.com/search?q="

In [5]:
search_word=["novels","supplements","mattresses","gaming-consoles","cameras"]

In [6]:
url=[]
for i in search_word:
    url.append(base_url+i)
print(url)

['https://www.flipkart.com/search?q=novels', 'https://www.flipkart.com/search?q=supplements', 'https://www.flipkart.com/search?q=mattresses', 'https://www.flipkart.com/search?q=gaming-consoles', 'https://www.flipkart.com/search?q=cameras']


In [7]:
pattern = re.compile(r'\d+')

In [9]:
df_list=[]
for i,j in zip(url,search_word):
    df_list.append(get_df(i,j))

<Response [200]>
Length of prod_id: 40
Length of prices: 40
Length of ratings: 40
Length of images: 40
Length of description: 40
Length of seller: 40
Length of seller_ratings: 40
Length of num_reviews: 40
Length of num_ratings: 40


<Response [200]>
Length of prod_id: 40
Length of prices: 40
Length of ratings: 40
Length of images: 40
Length of description: 40
Length of seller: 40
Length of seller_ratings: 40
Length of num_reviews: 40
Length of num_ratings: 40


<Response [200]>


KeyboardInterrupt: 

In [66]:
result_df=pd.concat(df_list, ignore_index=True)
result_df

,Search Word,Product Title,Prices,Rating,Description,Num of Ratings,Num of Reviews,Product Id,Images,Seller,Seller Ratings
0,novels,The Complete Novel of Sherlock Holmes,₹189,4.5,HighlightsLanguage: EnglishBinding: PaperbackP...,"9,498 Ratings",796 Reviews,9788175994317,[https://rukminim2.flixcart.com/image/416/416/...,NSPRetail,4.5
1,novels,1984,₹116,4.5,HighlightsLanguage: EnglishBinding: PaperbackP...,"15,006 Ratings",986 Reviews,9789354860461,[https://rukminim2.flixcart.com/image/416/416/...,Rajpalray,No rating
2,novels,Set Of 2 Best Selling (Combo) Story Books,₹256,4.2,HighlightsAuthor: paulo coelho & khhaled hosse...,165 Ratings,16 Reviews,RBKGA49CPZFERGGB,[https://rukminim2.flixcart.com/image/416/416/...,singlebooks,4.3
3,novels,Complete Series Of Pery Jackson(Set Of 5 Books)C,₹849,4.2,HighlightsAuthor: Percy Jackson1685 PagesLangu...,168 Ratings,16 Reviews,RBKG66KPSYYEEZG3,[https://rukminim2.flixcart.com/image/128/128/...,CreativeworldStore,3.8
4,novels,"Mindset (English, Paperback, Dr Dweck Carol)",₹210,4.4,HighlightsAuthor: Dr Dweck Carol320 PagesLangu...,203 Ratings,14 Reviews,RBKG5ZSQUSTAVGYS,[https://rukminim2.flixcart.com/image/416/416/...,RJBOOK3,3.7
...,...,...,...,...,...,...,...,...,...,...,...
195,cameras,Insta360 One X2 Sports and Action Camera,"₹34,990",4.4,HighlightsEffective Pixels: 05.7K,551 Ratings,52 Reviews,SAYFXB3FN5UCFTUP,[https://rukminim2.flixcart.com/image/128/128/...,RetailNet,4.7
196,cameras,FUJIFILM Instax Treasure Box Mini 11 Instant C...,"₹6,499",4.4,HighlightsExposure Mode: AutomaticView Finder:...,"1,449 Ratings",74 Reviews,INAG37FNHMJBV5JF,[https://rukminim2.flixcart.com/image/416/416/...,RetailNet,4.7
197,cameras,SONY Alpha ILCE-6100L APS-C Mirrorless Camera ...,"₹66,989",4.5,"HighlightsHigh-resolution 4K movie recording, ...",590 Ratings,91 Reviews,CAMFM67HBBUJWA9Y,[https://rukminim2.flixcart.com/image/128/128/...,SVPeripherals,4.8
198,cameras,NIKON Z 50 Mirrorless Camera Body with 16-50mm...,"₹87,990",4.5,HighlightsEdit videos on the fly (Shoot a clip...,348 Ratings,43 Reviews,DLLFHY8YCXP7WM32,[https://rukminim2.flixcart.com/image/128/128/...,SVPeripherals,4.8


In [67]:
result_df.to_csv('Flipkart Webstories .csv')

In [72]:
df_filtered = result_df[result_df['Images'].apply(len) >= 3]
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

,Search Word,Product Title,Prices,Rating,Description,Num of Ratings,Num of Reviews,Product Id,Images,Seller,Seller Ratings
0,novels,Complete Series Of Pery Jackson(Set Of 5 Books)C,₹849,4.2,HighlightsAuthor: Percy Jackson1685 PagesLangu...,168 Ratings,16 Reviews,RBKG66KPSYYEEZG3,[https://rukminim2.flixcart.com/image/128/128/...,CreativeworldStore,3.8
1,novels,The Diary Of A Young Girl | All Time Great Cla...,₹99,4.4,HighlightsAuthor: Sawan168 PagesLanguage: Engl...,60 Ratings,7 Reviews,RBKFY8H9RFNZKMMG,[https://rukminim2.flixcart.com/image/128/128/...,GOWOO,4.6
2,novels,Messed Up! But All For Love,₹178,4.3,HighlightsLanguage: EnglishBinding: PaperbackP...,"1,507 Ratings",105 Reviews,9789382665946,[https://rukminim2.flixcart.com/image/128/128/...,SrishtiPD,4.8
3,novels,Two Indian Girls - Book 1 of The Kanke Killin...,₹202,4.1,HighlightsLanguage: EnglishBinding: PaperbackP...,208 Ratings,15 Reviews,9798758994283,[https://rukminim2.flixcart.com/image/128/128/...,AuthorKinshuk,4.5
4,novels,Uzumaki (3-in-1 Deluxe Edition),₹380,3.9,HighlightsLanguage: EnglishBinding: PaperbackP...,376 Ratings,42 Reviews,9781421561325,[https://rukminim2.flixcart.com/image/128/128/...,harinovels,No rating
...,...,...,...,...,...,...,...,...,...,...,...
156,cameras,SONY Alpha ILCE-7M3K Full Frame Mirrorless Cam...,"₹1,41,489",4.5,Highlights4K HDR movie recording capability/4K...,419 Ratings,54 Reviews,DLLF7GBGWYNB5SZK,[https://rukminim2.flixcart.com/image/128/128/...,SVPeripherals,4.8
157,cameras,Insta360 One X2 Sports and Action Camera,"₹34,990",4.4,HighlightsEffective Pixels: 05.7K,551 Ratings,52 Reviews,SAYFXB3FN5UCFTUP,[https://rukminim2.flixcart.com/image/128/128/...,RetailNet,4.7
158,cameras,SONY Alpha ILCE-6100L APS-C Mirrorless Camera ...,"₹66,989",4.5,"HighlightsHigh-resolution 4K movie recording, ...",590 Ratings,91 Reviews,CAMFM67HBBUJWA9Y,[https://rukminim2.flixcart.com/image/128/128/...,SVPeripherals,4.8
159,cameras,NIKON Z 50 Mirrorless Camera Body with 16-50mm...,"₹87,990",4.5,HighlightsEdit videos on the fly (Shoot a clip...,348 Ratings,43 Reviews,DLLFHY8YCXP7WM32,[https://rukminim2.flixcart.com/image/128/128/...,SVPeripherals,4.8


In [1]:
df_filtered.to_csv('Filtered Flipkart Webstories .csv')

NameError: name 'df_filtered' is not defined

In [74]:
# Convert dataframe into json object 
json = df_filtered.to_json() 
  
# Print json object 
print(json)

{"Search Word":{"0":"novels","1":"novels","2":"novels","3":"novels","4":"novels","5":"novels","6":"novels","7":"novels","8":"novels","9":"novels","10":"novels","11":"supplements","12":"supplements","13":"supplements","14":"supplements","15":"supplements","16":"supplements","17":"supplements","18":"supplements","19":"supplements","20":"supplements","21":"supplements","22":"supplements","23":"supplements","24":"supplements","25":"supplements","26":"supplements","27":"supplements","28":"supplements","29":"supplements","30":"supplements","31":"supplements","32":"supplements","33":"supplements","34":"supplements","35":"supplements","36":"supplements","37":"supplements","38":"supplements","39":"supplements","40":"supplements","41":"supplements","42":"supplements","43":"supplements","44":"supplements","45":"supplements","46":"supplements","47":"supplements","48":"supplements","49":"supplements","50":"supplements","51":"mattresses","52":"mattresses","53":"mattresses","54":"mattresses","55":"ma